In [1]:
%load_ext autoreload
%autoreload 2

# add logging to a file in case we disconnect from the notebook
import sys
import logging

nblog = open("nb.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

from movie_metadata_table import MovieMetadataTable

import sys
sys.path.append("../algorithms")
from deepfm import DeepFM # type: ignore

In [3]:
movie_metadata_table = MovieMetadataTable(
    movie_ids_file="../data/movie_ids.json",
    movie_data_vectorized_file="../data/vectorizing/movie_data_vectorized.csv",
    nlp_vectors_file="../data/vectorizing/nlp_vectors.pt",
)

In [4]:
import json
import random

random.seed(8888)

# get movie ids, clipping off movie ids that are not in the movie tensor
all_movie_ids = json.load(open("../data/movie_ids.json"))[:movie_metadata_table.movie_tensor.shape[0]]
movie_metadata_isna = movie_metadata_table.movie_metadata.isna().any(axis=1)
all_movie_ids = [all_movie_ids[i] for i in range(len(all_movie_ids)) if not movie_metadata_isna[i]]
random.shuffle(all_movie_ids)

In [5]:
ratings = pd.read_csv("../data/ratings_export.csv")

In [6]:
train_movie_ids = all_movie_ids[:9000]
test_movie_ids = all_movie_ids[9000:10000]

ratings = ratings.dropna()
train_ratings = ratings[ratings['movie_id'].isin(train_movie_ids)]
test_ratings = ratings[ratings['movie_id'].isin(test_movie_ids)]

# Identify users who are in both
avail_train_users = set(train_ratings['user_id'].unique())
avail_test_users = set(test_ratings['user_id'].unique())
avail_users = avail_train_users.intersection(avail_test_users)

train_ratings = train_ratings[train_ratings['user_id'].isin(avail_users)]
test_ratings = test_ratings[test_ratings['user_id'].isin(avail_users)]

train_movie_ids_filtered = train_ratings['movie_id'].unique()
test_movie_ids_filtered = test_ratings['movie_id'].unique()

print("Number of available users:", len(avail_users))
print("Number of training movies:", len(train_movie_ids_filtered))
print("Number of testing movies:", len(test_movie_ids_filtered))

device = "cuda" if torch.cuda.is_available() else "cpu"
user_vector_size = 16
user_id_to_index = {user_id: i for i, user_id in enumerate(avail_users)}

Number of available users: 6280
Number of training movies: 8976
Number of testing movies: 999


In [13]:
user_embedding_table = nn.Embedding(len(avail_users), user_vector_size).to(device)
deepfm = DeepFM(
    movie_metadata_table.movie_vector_size,
    user_vector_size,
    num_dense_movie_embeddings=1,
    num_dense_user_embeddings=1,
    dense_embedding_size=16,
    mlp_sizes=[32, 1],
).to(device)

In [ ]:
torch.save(user_embedding_table.state_dict(), "user_embeddings.pt")
torch.save(deepfm.state_dict(), "deepfm.pt")

In [9]:
print(len(movie_metadata_table.movie_ids), movie_metadata_table.movie_tensor.shape)

285961 torch.Size([270422, 231])


In [14]:
import tqdm.notebook as tqdm_notebook

# note: if we want to get a massive speedup,
# we can probably use a sparse optimization scheme somehow
optim = torch.optim.Adam([
    *deepfm.parameters(),
    *user_embedding_table.parameters()
], lr=0.001)

# determine how to give a reward
# we will just give a reward if the user rated the movie >= 7/10

loss_type = "mse"
reward_rating_cutoff = 7

avg_reward_per_movie_numer = torch.zeros(len(train_movie_ids), device=device)
avg_reward_per_movie_denom = torch.zeros(len(train_movie_ids), device=device)

# iterate over ratings in random batches
indexes = torch.randperm(len(train_ratings))
batch_size = 1024

# Store the reward history.
epochs = 200
loss_history = []
accuracy_history = []
baseline_history = [] # note(michael): should be a constant but i'm logging it every time for my sanity

for epoch in range(epochs):
    epoch_loss_total = 0
    epoch_corrects = 0
    epoch_correct_baseline = 0
    epoch_seen = 0
    for batch_start in (pbar := tqdm_notebook.tqdm(range(0, len(indexes), batch_size), desc='Epoch ' + str(epoch + 1))):
        batch = train_ratings.iloc[indexes[batch_start:batch_start + batch_size]]

        movie_slugs = [str(x) for x in batch['movie_id'].values]
        train_user_ids = batch['user_id'].values
        ratings_ = batch['rating_val'].values

        user_indices = torch.tensor([user_id_to_index[user_id] for user_id in train_user_ids], device=device)
        user_vectors = user_embedding_table(user_indices)
        movie_vectors = movie_metadata_table(movie_slugs).to(device)
        movie_average_ratings = movie_vectors[:, 2].to(device)
        
        assert not movie_vectors.isnan().any(), "NaN movie vectors found."

        # add a bias if the average movie rating is greater than 7
        predictions = torch.sigmoid(deepfm(movie_vectors.float(), user_vectors.float()).squeeze(-1)) # + 1.0 * (movie_vectors[..., 2] > 7))
        rewards = torch.tensor(ratings_ >= reward_rating_cutoff, device=device, dtype=torch.float32)

        assert not torch.any(rewards.isnan())

        if loss_type == 'mse':
            # resembles learning q function
            loss = F.mse_loss(predictions, rewards)
        elif loss_type == 'binary_crossentropy':
            # loosely resembles policy gradient
            loss = F.binary_cross_entropy_with_logits(predictions, rewards.float())

        optim.zero_grad()
        loss.backward()
        optim.step()
        
        epoch_corrects += ((predictions >= 0.5) == rewards).sum()
        
        movie_pos_in_tracking_vector = [train_movie_ids.index(slug) for slug in movie_slugs]
        avg_reward_per_movie_numer[movie_pos_in_tracking_vector] += rewards
        avg_reward_per_movie_denom[movie_pos_in_tracking_vector] += 1
        baseline_reward_predictions = (avg_reward_per_movie_numer/avg_reward_per_movie_denom)[movie_pos_in_tracking_vector]
        baseline_corrects = (baseline_reward_predictions >= 0.5) == rewards
        # baseline_corrects = (movie_average_ratings > reward_rating_cutoff) == rewards
        epoch_correct_baseline += baseline_corrects.sum()
        epoch_loss_total += loss.item() * len(user_indices)
        epoch_seen += len(user_indices)

        pbar.set_postfix(loss=epoch_loss_total / epoch_seen, accuracy=(epoch_corrects / epoch_seen).item(), baseline=(epoch_correct_baseline / epoch_seen).item())
    
    loss_history.append(epoch_loss_total / epoch_seen)
    accuracy_history.append((epoch_corrects / epoch_seen).item())
    baseline_history.append((epoch_correct_baseline / epoch_seen).item())
    
    print(f"Training @ {epoch+1}:", dict(loss=epoch_loss_total / epoch_seen, accuracy=(epoch_corrects / epoch_seen).item(), baseline=(epoch_correct_baseline / epoch_seen).item()))
    evaluate(test_ratings, test_movie_ids, epoch+1)


Epoch 1:   0%|          | 0/363 [00:00<?, ?it/s]

Training @ 1: {'loss': 0.22195157218148026, 'accuracy': 0.6444095373153687, 'baseline': 0.7546234130859375}


Evaluating:   0%|          | 0/39 [00:00<?, ?it/s]

Validation @ 1: {'loss': 0.2853745449340099, 'accuracy': 0.4880152940750122, 'baseline': 0.737946093082428}


Epoch 2:   0%|          | 0/363 [00:00<?, ?it/s]

...

Validation @ 199: {'loss': 0.39495378102514206, 'accuracy': 0.5649538636207581, 'baseline': 0.7425236701965332}


Epoch 200:   0%|          | 0/363 [00:00<?, ?it/s]

Training @ 200: {'loss': 0.10038216791309446, 'accuracy': 0.8878083825111389, 'baseline': 0.7502912878990173}


Evaluating:   0%|          | 0/39 [00:00<?, ?it/s]

Validation @ 200: {'loss': 0.3937053849096614, 'accuracy': 0.5661610960960388, 'baseline': 0.7291682362556458}


## calculate validation results

In [11]:
def evaluate(test_ratings, test_movie_ids, epoch_num):
    epoch_correct_baseline = 0
    epoch_corrects = 0
    epoch_seen = 0
    epoch_loss_total = 0
    indexes = torch.randperm(len(test_ratings))
    
    avg_reward_per_movie_numer = torch.zeros(len(test_movie_ids), device=device)
    avg_reward_per_movie_denom = torch.zeros(len(test_movie_ids), device=device)
    
    for batch_start in (pbar := tqdm_notebook.tqdm(range(0, len(indexes), batch_size), desc='Evaluating')):
        batch = test_ratings.iloc[indexes[batch_start:batch_start + batch_size]]

        movie_slugs = [str(x) for x in batch['movie_id'].values]
        user_ids = batch['user_id'].values
        ratings_ = batch['rating_val'].values

        with torch.no_grad():
            user_indices = torch.tensor([user_id_to_index[user_id] for user_id in user_ids], device=device)
            user_vectors = user_embedding_table(user_indices)
            movie_vectors = movie_metadata_table(movie_slugs).to(device)
            movie_average_ratings = movie_vectors[:, 2].to(device)

            assert not movie_vectors.isnan().any(), "NaN movie vectors found."

            predictions = torch.sigmoid(deepfm(movie_vectors.float(), user_vectors.float()).squeeze(-1))
            rewards = torch.tensor(ratings_ >= reward_rating_cutoff, device=device, dtype=torch.float32)
            
            # calculate loss, but do not use for optimization!!
            if loss_type == 'mse':
                # resembles learning q function
                loss = F.mse_loss(predictions, rewards)
            elif loss_type == 'binary_crossentropy':
                # loosely resembles policy gradient
                loss = F.binary_cross_entropy_with_logits(predictions, rewards.float())

        epoch_corrects += ((predictions >= 0.5) == rewards).sum()

        movie_pos_in_tracking_vector = [test_movie_ids.index(slug) for slug in movie_slugs]
        avg_reward_per_movie_numer[movie_pos_in_tracking_vector] += rewards
        avg_reward_per_movie_denom[movie_pos_in_tracking_vector] += 1
        baseline_reward_predictions = (avg_reward_per_movie_numer/avg_reward_per_movie_denom)[movie_pos_in_tracking_vector]
        baseline_corrects = (baseline_reward_predictions >= 0.5) == rewards
        # baseline_corrects = (movie_average_ratings > reward_rating_cutoff) == rewards
        epoch_correct_baseline += baseline_corrects.sum()
        epoch_loss_total += loss.item() * len(user_indices)
        epoch_seen += len(user_indices)

        pbar.set_postfix(loss=epoch_loss_total / epoch_seen, accuracy=(epoch_corrects / epoch_seen).item(), baseline=(epoch_correct_baseline / epoch_seen).item())
        
    # print data explicitly in case tqdm messes up
    print(f"Validation @ {epoch_num}:", dict(loss=epoch_loss_total / epoch_seen, accuracy=(epoch_corrects / epoch_seen).item(), baseline=(epoch_correct_baseline / epoch_seen).item()))


In [66]:
evaluate(test_ratings, test_movie_ids, 100)

Evaluating:   0%|          | 0/39 [00:00<?, ?it/s]

Validation @ 100: {'loss': 0.2795397595012238, 'accuracy': 0.521793782711029, 'baseline': 0.7370909452438354}


In [28]:
epoch_loss_total/epoch_seen

0.11914060991449937

In [29]:
epoch_corrects/epoch_seen

tensor(0.8453, device='cuda:0')

In [31]:
epoch_correct_baseline/epoch_seen

tensor(0.7505, device='cuda:0')